In [4]:
%load_ext autoreload
%autoreload 2

In [3]:
import os

os.chdir('..')
os.getcwd()

'/home/member/Documents/Emotion/pl-ibug-facetrack'

In [2]:
import torch
from torch.utils.data import Dataset, ConcatDataset, DataLoader
from torch.utils.data.sampler import BatchSampler, SequentialSampler, Sampler
from functools import partial

class MyBatchSampler(BatchSampler):
    def __init__(self, sampler, batch_size: int, drop_last: bool, chunk_lens: list) -> None:
        super().__init__(sampler, batch_size, drop_last)
        self.chunk_lens = chunk_lens
        
    def __iter__(self):        
        offset = 0
        for chunk_len in chunk_lens:
            batch = []
            for idx in range(offset, offset+chunk_len):#self.sampler: # range(37)
                batch.append(idx)
                if len(batch) == self.batch_size:
                    yield batch
                    batch = []
            if len(batch) > 0 and not self.drop_last:
                yield batch
            offset += chunk_len

    def __len__(self):
        if self.drop_last:
            l = sum([chunk_len//self.batch_size for chunk_len in chunk_lens])
            return l  # type: ignore
        else:
            return sum([(chunk_len + self.batch_size - 1)//self.batch_size for chunk_len in chunk_lens]) # type: ignore

In [79]:
from pathlib import Path

root = Path("./data/")

paths = [
    str(p)
    for p in list(root.glob("300VW/*/")) + list(root.glob("CONFER/**/stitched"))
]

print(f"Found n={len(paths)} paths")

Found n=143 paths


In [97]:
from dataset import FaceTrackDatasetFolder

datasets = [FaceTrackDatasetFolder(path) for path in paths]

chunk_lens = list(map(len, datasets))
dataset_combine = ConcatDataset(datasets)

In [102]:
my_batch_sampler = MyBatchSampler(batch_size=256, sampler=SequentialSampler(dataset_combine), chunk_lens=chunk_lens, drop_last=False)
dataloader = DataLoader(dataset_combine, batch_sampler=my_batch_sampler)